# ATMS 523
## Module 4 Notebook 1
## Wrangling time series

### Combining datasets with pandas

Here we will load a variety of climate datasets, and use pandas to unify the datasets for further analysis.

In [ ]:
import pandas as pd

enso = pd.read_csv('https://www.esrl.noaa.gov/psd/data/correlation/censo.data',delim_whitespace=True,header=None,skiprows=1,skipfooter=2, engine='python')
pdo = pd.read_csv('https://www.esrl.noaa.gov/psd/data/correlation/pdo.data',delim_whitespace=True,header=None,skiprows=1,skipfooter=14, engine='python')
nao = pd.read_csv('https://www.esrl.noaa.gov/psd/data/correlation/nao.data',delim_whitespace=True,header=None,skiprows=1,skipfooter=3, engine='python')
ao = pd.read_csv('https://www.esrl.noaa.gov/psd/data/correlation/ao.data',delim_whitespace=True,header=None,skiprows=1,skipfooter=3, engine='python')

Take a look at the dataframes, note how they are formatted and if there are any issues with the values (missing data, etc.).

In [ ]:
enso


Now that we have read in the data, it's time to get them into a format that we can use with pandas.  Time series should be a single column, with a date column with a value of each index in each row.

We'll approach this by creating a new data frame and reformatting the 12 x nyears array to a 1 x nobservations array corresponding with each date.

Let's start with ENSO.

In [ ]:
enso_new=pd.DataFrame()


In [ ]:
enso_new

Let's create an ENSO 'Date' column that has a Datetime Index.  The pandas DatetimeIndex can be used to generate dates with a given frequency.

In [ ]:
enso_new['Date']=pd.date_range(start=pd.datetime(1948,1,1),end=pd.datetime(2022,12,1),freq="MS")

enso_new = enso_new.set_index('Date')

In [ ]:
enso_new

Now, let's stuff the data into our array.  We can use the stack() command to take the 2D array and create a 1D column.

In [ ]:
len(enso.loc[:,1:].stack().values)

In [ ]:
enso_new['ENSO']=enso.loc[:,1:].stack().values

How does it look?

In [ ]:
enso_new

In [ ]:
enso_new.plot()

## In-class exercise, do this for all of the 4 climate indicies.

Be sure to check the date ranges of your indicies by carefully inspecting the dataframes!

In [ ]:
nao_new=pd.DataFrame()
pdo_new=pd.DataFrame()
ao_new=pd.DataFrame()

nao_new['Date'] = pd.date_range(start=pd.datetime(1948,1,1),end=pd.datetime(2022,12,1),freq="MS")
pdo_new['Date'] = pd.date_range(start=pd.datetime(1948,1,1),end=pd.datetime(2022,12,1),freq="MS")
ao_new['Date'] = pd.date_range(start=pd.datetime(1950,1,1),end=pd.datetime(2022,12,1),freq="MS")

nao_new = nao_new.set_index('Date')
pdo_new = pdo_new.set_index('Date')
ao_new = ao_new.set_index('Date')

nao_new['NAO']=nao.loc[:,1:].stack().values
pdo_new['PDO']=pdo.loc[:,1:].stack().values
ao_new['AO']=ao.loc[:,1:].stack().values

In [ ]:
pdo_new

In [ ]:
%pylab inline
nao_new['NAO'].plot()
plt.ylim([-3,3])
plt.xlim(['11-01-2018','01-01-2020'])

# Merging dataframes

Now that we have these indicies, we can join them together into one dataset.  We can use the pd.merge() command to do this for each of our dataframes one by one.  We need to tell pandas how to do the merge, and we do that by specifying the left_index and right_index, where left and right are the data frames given first and second in the pd.merge command.

In [ ]:
newdf_all = pd.merge(enso_new,pdo_new, left_index=True, right_index=True)

In [ ]:
newdf_all

In [ ]:
import numpy as np
newdf_all['PDO'][newdf_all['PDO'] <= -9.9] = np.nan
newdf_all.plot()

Rinse and repeat until all 4 columns are included in your new dataframe.

In [ ]:
newdf_all['ENSO'][newdf_all['ENSO'] <= -9.9] = np.nan
newdf_all.plot()

In [ ]:
newdf_all = pd.merge(newdf_all,nao_new, left_index=True, right_index=True)
newdf_all['NAO'][newdf_all['NAO'] <= -99.9] = np.nan

In [ ]:
newdf_all = pd.merge(newdf_all,ao_new, left_index=True, right_index=True)
newdf_all['AO'][newdf_all['AO'] <= -99.9] = np.nan

newdf_all.plot()

In [ ]:
newdf_all

Let's plot them against each other


In [ ]:

plt.plot(newdf_all['ENSO'].values,newdf_all['PDO'].values,'.r')

Or do it all together in one swoop.

In [ ]:
# Seaborn visualization library
import seaborn as sns
# Create the default pairplot
sns.pairplot(newdf_all)


Now let's bring in another dataset, the NASA GISS observed temperature record.  How does it match up with the climate indicies above?

In [ ]:
!wget https://data.giss.nasa.gov/pub/gistemp/gistemp250_GHCNv4.nc.gz

In [ ]:
%matplotlib inline
import xarray as xr
import pandas as pd
import cartopy.crs as ccrs

In [ ]:
data = xr.open_dataset('gistemp250_GHCNv4.nc.gz')
data
   

In [ ]:
ax = plt.axes(projection=ccrs.Orthographic(-80, 35))
data['tempanomaly'].polyfit(dim='time',deg=1).isel(degree=1)['polyfit_coefficients'].plot(ax=ax, transform=ccrs.PlateCarree(), vmin=-0.5, vmax=0.5, cmap='RdBu_r')
ax.set_global(); ax.coastlines(); ax.set_title('GISTEMPv4 temperature trend (deg/year)')

Here we will introduce the sel command to find time and space subsets of our dataset.  Let's pull out the observed temperature trends at the point closest to Champaign-Urbana and examine the relationships with climate indicies.

In [ ]:
temp = pd.DataFrame()

temp.index = data.time-pd.Timedelta(14, unit="D")
temp['obsT']=data['tempanomaly'].sel(lat=40.,lon=-88., method='nearest')

newdf_all = pd.merge(newdf_all,temp,left_index=True, right_index=True)


In [ ]:
temp

In [ ]:
newdf_all

Let's plot!

In [ ]:
%pylab inline
import matplotlib as mpl

cmap = cm.get_cmap('PiYG', 12)    # 11 discrete colors
norm = mpl.colors.Normalize(vmin=1, vmax=13)

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(nrows=2, ncols=2, figsize=(11,11))

ax1.scatter(newdf_all['ENSO'].values,newdf_all['obsT'].values,c=newdf_all.index.month,cmap=cmap,norm=norm)
ax1.set_title('ENSO')
ax1.set_xlabel('ENSO index')
ax1.set_ylabel('Monthly GISTEMP anomaly')
ax1.set_ylim([-10,10])
ax1.set_xlim([-5,5])

ax2.scatter(newdf_all['NAO'].values,newdf_all['obsT'].values,c=newdf_all.index.month,cmap=cmap,norm=norm)
ax2.set_title('NAO')
ax2.set_xlabel('NAO index')
ax2.set_ylabel('Monthly GISTEMP anomaly')
ax2.set_ylim([-10,10])
ax2.set_xlim([-5,5])

ax3.scatter(newdf_all['AO'].values,newdf_all['obsT'].values,c=newdf_all.index.month,cmap=cmap,norm=norm)
ax3.set_title('AO')
ax3.set_xlabel('AO index')
ax3.set_ylabel('Monthly GISTEMP anomaly')
ax3.set_ylim([-10,10])
ax3.set_xlim([-5,5])


im=ax4.scatter(newdf_all['PDO'].values,newdf_all['obsT'].values,c=newdf_all.index.month,cmap=cmap,norm=norm)
ax4.set_title('PDO')
ax4.set_xlabel('PDO index')
ax4.set_ylabel('Monthly GISTEMP anomaly')
ax4.set_ylim([-10,10])
ax4.set_xlim([-5,5])

#fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([1.05, 0.15, 0.02, 0.7])
cbar=fig.colorbar(im, cax=cbar_ax, ticks=1+np.arange(13), norm=norm)
cbar.ax.set_yticklabels(['J','F','M','A','M','J','J','A','S','O','N','D',' '])  # vertically oriented colorbar

plt.tight_layout()